In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import wrangle as wr

from env import get_db_url

# Acquiring Data:
- Acquiring data for *single family homes* that have transaction dates *during the year of 2017.*

In [2]:
query_df = wr.get_zillow_data()

In [3]:
#Verifying that the dataset meets the rubric requirement of being 52,441 homes long:
query_df.shape[0]

52441

## Wrangling Data:
- I readjusted my wrangle function to exclude any homes that are over 1.5 million dollars. 
- This, paired with the other operations and cleaning carried out in the wrangle script, leave 95.4% of the data remaining. 

In [9]:
#Acquiring CLEANED data, beyond the initial SQL query:
df = wr.wrangle_zillow()

In [10]:
#Verifying how much of the data was cleaned out (less than 5%):
df.shape[0] / query_df.shape[0]

0.9540435918460747